In [9]:
import torch 
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np

from API_utils.dataset_FEGS import API_FEGS_Class


In [10]:
DATASET_PATH = "C:/Users/asus/Desktop/API/dataset/one_to_one.xls"
DATASET_MAT_PATH = "C:/Users/asus/Desktop/API/dataset/one_to_one.mat"
TEST_DATASET_PATH = "C:/Users/asus/Desktop/API/dataset/test.xlsx"
TEST_DATASET_MAT_PATH = "C:/Users/asus/Desktop/API/dataset/test.mat"
SAVE_MODEL_PATH = "C:/Users/asus/Desktop/API/Model/"
DEVICE= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCH = 1000
lr=0.0001

In [11]:
print(DEVICE)

cuda:0


In [12]:
train_data = API_FEGS_Class(DATASET_PATH,DATASET_MAT_PATH,'abc')
test_data = API_FEGS_Class(TEST_DATASET_PATH,TEST_DATASET_MAT_PATH,'test')

In [13]:
class NeuralNet(nn.Module):
    def __init__(self, lrate = 0.001, loss_fn = nn.CrossEntropyLoss()):
        super(NeuralNet, self).__init__()


        self.conv1 = nn.Sequential(nn.Conv1d(1,8,kernel_size=1),
                                   nn.BatchNorm1d(8),
                                   nn.LeakyReLU()
        )
        self.conv2 = nn.Sequential(nn.Conv1d(8,16,kernel_size=1),
                                   nn.BatchNorm1d(16),
                                   nn.LeakyReLU()
        )
        self.conv3 = nn.Sequential(nn.Conv1d(1,8,kernel_size=3,stride=2),
                                   nn.BatchNorm1d(8),
                                   nn.LeakyReLU()
        )
        self.conv4 = nn.Sequential(nn.Conv1d(8,16,kernel_size=3,stride=2),
                                   nn.BatchNorm1d(16),
                                   nn.LeakyReLU()
        )

        self.conv5 = nn.Sequential(nn.Conv1d(16,8,kernel_size=3,stride=2),
                                   nn.BatchNorm1d(8),
                                   nn.MaxPool1d(kernel_size=2),
                                   nn.LeakyReLU()
        )
        self.conv6 = nn.Sequential(nn.Conv1d(8,1,kernel_size=3,stride=2),
                                   nn.BatchNorm1d(1),
                                   nn.MaxPool1d(kernel_size=2),
                                   nn.LeakyReLU()
        )

        self.conv7 = nn.Linear(12,2)
        self.conv8 = nn.Linear(2,1)

        self.loss_fn = loss_fn
        self.lrate=lrate        
        self.optimizer=optim.SGD(self.parameters(),self.lrate,momentum=0.9)
    def forward(self, rna, protein):
        """Performs a forward pass through your neural net (evaluates f(x)).

        @param x: an (N, in_size) Tensor
        @return y: an (N, out_size) Tensor of output from the network
        """
        '''
        extractor for protein 
        '''
        
        rna = self.conv1(rna)
        rna = self.conv2(rna)
        # print(rna.shape)
        protein = self.conv3(protein)
        
        protein = self.conv4(protein)
        # print(protein.shape)
        x = torch.cat((rna,protein),dim=2)
        x = self.conv5(x)
        x = self.conv6(x)
        # print(x.shape)
        x = self.conv7(x)
        x = self.conv8(x)
        x = torch.squeeze(x,dim=0)
        return x


In [14]:
train_data_loader = DataLoader(train_data,batch_size=1,shuffle=True)
test_data_loader = DataLoader(test_data,batch_size=1,shuffle=True)
Net = NeuralNet().to(DEVICE)
optimizer = optim.SGD(params=Net.parameters(),lr=lr,momentum=0.9)
loss_fn = nn.BCEWithLogitsLoss()


In [15]:
def evaluate(model_path,test_data_loader,device=DEVICE):
    count=0
    acc = 0
    model = torch.load(model_path)
    for idx, data in enumerate(test_data_loader):
        api_input, api_label = data
        count+=1
        rna_input = api_input[:,0:64]
        # print(rna_input)
        protein_input = api_input[:,64:]
        rna_input = torch.unsqueeze(rna_input,dim=0)
        protein_input = torch.unsqueeze(protein_input,dim=0)
        #api_label = torch.unsqueeze(api_label,dim=0)
        rna_input = rna_input.to(dtype=torch.float32).to(DEVICE)
        protein_input = protein_input.to(dtype=torch.float32).to(DEVICE)
        output = model(rna_input,protein_input).to(DEVICE)
        if(torch.round(torch.sigmoid(output))==api_label[0]):
            acc+=1

    
    print("Accuracy",acc/count)

In [16]:
for epoch in range(EPOCH):
    for data in tqdm(train_data_loader):
        api_input, api_label = data
        api_input = api_input.to(DEVICE)
        api_label = api_label.to(DEVICE)
        rna_input = api_input[:,0:64]
        # print(api_input.shape)
        protein_input = api_input[:,64:]
        rna_input = torch.unsqueeze(rna_input,dim=0)
        protein_input = torch.unsqueeze(protein_input,dim=0)

        api_label = torch.unsqueeze(api_label,dim=0).to(dtype=torch.float32)
        rna_input = rna_input.to(dtype=torch.float32)
        protein_input = protein_input.to(dtype=torch.float32)
        output = Net(rna_input,protein_input)
        break
        # print(output)
        optimizer.zero_grad()    
        Loss = loss_fn(output, api_label)  
        Loss.backward()  
        optimizer.step()

    print("Loss",Loss.item())
    save_path = SAVE_MODEL_PATH+f'Epoch={epoch}_lr={lr}.pth'
    torch.save(Net, save_path)
    evaluate(SAVE_MODEL_PATH+f'Epoch={epoch}_lr={lr}.pth',test_data_loader)
    
torch.save(Net, SAVE_MODEL_PATH+f'Epoch={epoch}_lr={lr}_final.pth')
print('CSX')

  0%|          | 0/1160 [00:00<?, ?it/s]

torch.Size([1, 1, 578])
tensor([[[0.9056, 0.9071, 0.8994, 0.9005, 0.9048, 0.8931, 0.8988, 0.8948,
          0.8996, 0.9000, 0.9021, 0.9028, 0.8973, 0.8966, 0.8970, 0.9013,
          0.9056, 0.9055, 0.8978, 0.8973, 0.8963, 0.8977, 0.9029, 0.9009,
          0.8961, 0.9015, 0.8975, 0.9050, 0.9006, 0.9047, 0.8939, 0.8958,
          0.8970, 0.8999, 0.9022, 0.8977, 0.8955, 0.9017, 0.8968, 0.9042,
          0.9036, 0.8966, 0.9087, 0.8998, 0.8992, 0.8959, 0.8985, 0.9076,
          0.8978, 0.8958, 0.8953, 0.8963, 0.8972, 0.9039, 0.8954, 0.9021,
          0.9026, 0.9111, 0.8955, 0.9036, 0.8966, 0.8995, 0.9032, 0.9031,
          0.8965, 0.8997, 0.8949, 0.8978, 0.8959, 0.9032, 0.8943, 0.9038,
          0.9046, 0.8957, 0.8970, 0.8979, 0.8941, 0.8988, 0.8978, 0.8953,
          0.8967, 0.8953, 0.9021, 0.8985, 0.9059, 0.9001, 0.9051, 0.9010,
          0.9023, 0.8950, 0.8954, 0.8981, 0.9055, 0.9043, 0.8954, 0.8953,
          0.8949, 0.9029, 0.8963, 0.9102, 0.8958, 0.8983, 0.9064, 0.8982,
          0.89

NameError: name 'Loss' is not defined

In [ ]:
model=torch.load(SAVE_MODEL_PATH+f'Epoch={epoch}_lr={lr}_final.pth')

In [ ]:

acc=0
count=0
for idx, data in enumerate(test_data_loader):
    api_input, api_label = data
    count+=1
    rna_input = api_input[:,0:64]
    protein_input = api_input[:,64:909]
    rna_input = torch.unsqueeze(rna_input,dim=0)
    protein_input = torch.unsqueeze(protein_input,dim=0)
    #api_label = torch.unsqueeze(api_label,dim=0)
    rna_input = rna_input.to(dtype=torch.float32)
    protein_input = protein_input.to(dtype=torch.float32)
    output = model(rna_input,protein_input)
    if(torch.argmax(output)==api_label[0]):
        acc+=1

    
print(acc/count)

